In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from cil.framework import ImageData, ImageGeometry, AcquisitionData, AcquisitionGeometry
from cil.utilities.display import show2D, show_geometry
from cil.plugins.astra import ProjectionOperator
from cil.plugins.astra.processors import FBP

In [ ]:
def create_grid(shape = (256, 64, 256)):
    # Create a circular grid phantom in a volume with shape (X, Y, Z)  
    volume = np.zeros(shape, dtype=np.float32)

    margin = 16
    center_x, center_z = shape[0] // 2, shape[2] // 2
    radius = min(center_x, center_z) - margin
    edge_thickness = 1

    z_vals = np.arange(margin, shape[0] - margin)
    x_vals = np.arange(margin, shape[2] - margin)
    zz, xx = np.meshgrid(z_vals, x_vals, indexing='ij')  

    r2 = (xx - center_x)**2 + (zz - center_z)**2
    cylinder_mask = r2 <= radius**2
    cylinder_edge_mask = (r2 >= (radius - edge_thickness)**2) & (r2 <= (radius + edge_thickness)**2)

    grid_spacing = 16

    for zi, z in enumerate(z_vals):
        for xi, x in enumerate(x_vals):
            if not cylinder_mask[zi, xi]:
                continue
            if (x % grid_spacing == 0) or (z % grid_spacing == 0):
                volume[z, margin:shape[1] - margin, x] = 1.0


    for yi in range(margin, shape[1] - margin):
        for zi, z in enumerate(z_vals):
            for xi, x in enumerate(x_vals):
                if cylinder_edge_mask[zi, xi]:
                    volume[z, yi, x] = 1.0


    ig = ImageGeometry(voxel_num_x=shape[0], voxel_num_y=shape[1], voxel_num_z=shape[2])
    grid = ImageData(volume, geometry=ig)

    return grid, ig


We start by simulating a flat circular mesh grid in a 3D volume

In [ ]:
# call the create_grid function to get a grid and its image geometry
grid, ig = create_grid()

# then show what the volume looks like from 3 different directions
ax = show2D(grid,
       slice_list=[
           (0, 100),  
           (1, grid.shape[1]//2),  
           (2, 100),
       ],
       num_cols=3,
       size=(15,5))


We simulate the projections which would be generated by the grid in a parallel beam CT geometry setup

In [ ]:
# first we create an acquisition geometry which will describe the projection data
ag = AcquisitionGeometry.create_Parallel3D()\
    .set_angles(np.arange(0,360))\
    .set_panel([256, 256])
ag.dimension_labels = ('vertical', 'angle','horizontal')

# then we use a projection operator to simulate the projections
A = ProjectionOperator(ig, ag)
proj = A.direct(grid)
ag_slice_list = [('angle', 0),('angle',45), ('angle',90), ('angle',135), ('angle',180)]
show2D(proj,
       slice_list = ag_slice_list, 
       num_cols=5,
       fix_range=(0,25))

Notice that the projections have quite different levels of intensity at different projections as we look through the long and short plane of the flat sample.

Let's see what it looks like if we try to reconstruct the grid from these projections.

In [ ]:
# Run the reconstruction
recon = FBP(image_geometry=None, acquisition_geometry=ag)(proj)

slice_list = [('vertical','centre'), ('horizontal_y',int(recon.shape[2]/2)), ('horizontal_x',int(recon.shape[1]/2))]
show2D(recon,
       slice_list=slice_list,
       num_cols=3)

In [ ]:
plt.plot(recon.array[:, 128, 128])

We notice some strange artefacts in the reconstruction. This is because of the un-even absorption we noticed in the projections.

A common solution to these kind of artefacts is to tilt the sample and rotation axis to give a more uniform absorption profile, this setup is called laminography. We can simulate the laminography setup by projecting the original grid volume with a tilted rotation axis

In [ ]:
# first rotate the grid volume onto the z-y plane
grid_tilt = np.rot90(grid.array, k=1, axes=(0, 1))
ig = ImageGeometry(voxel_num_x=grid_tilt.shape[2], voxel_num_y=grid_tilt.shape[1], voxel_num_z=grid_tilt.shape[0])
grid_tilt = ImageData(grid_tilt, geometry=ig)

# next we define a tilt around the x-axis
tilt = 30 # degrees
tilt_rad = np.deg2rad(tilt)
tilt_axis = np.array([1, 0, 0])
beam_direction = np.array([0, 1, 0])
rotation_axis = np.array([0, 0, 1]) # untilted rotation axis
rotation_matrix = Rotation.from_rotvec(tilt_rad * tilt_axis)
tilted_rotation_axis = rotation_matrix.apply(rotation_axis)

# we recreate the acquisition geometry with a tilted axis
ag = AcquisitionGeometry.create_Parallel3D(rotation_axis_direction=tilted_rotation_axis)\
    .set_angles(np.arange(0,360))\
    .set_panel([256, 256])
ag.dimension_labels = ('vertical', 'angle','horizontal')

# then we use a projection operator to simulate the projections again
A = ProjectionOperator(ig, ag)
proj = A.direct(grid_tilt)
ag_slice_list = [('angle', 0),('angle',45), ('angle',90), ('angle',135), ('angle',180)]
show2D(proj,
       slice_list = ag_slice_list, 
       num_cols=5,
       fix_range=(0,25))

The projections now have very uniform attenuation. Let's try reconstructing this dataset

In [ ]:
recon_tilt = FBP(image_geometry=None, acquisition_geometry=ag)(proj)

slice_list = [('vertical','centre'), ('horizontal_x',int(recon_tilt.shape[1]/2)), ('horizontal_y',int(recon_tilt.shape[2]/2))]
show2D(recon_tilt,
       slice_list=slice_list,
       num_cols=3)

In [ ]:
for tilt in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]:
    tilt_rad = np.deg2rad(tilt)

    rotation_matrix = Rotation.from_rotvec(tilt_rad * tilt_axis)
    tilted_rotation_axis = rotation_matrix.apply(rotation_axis)

    ag.config.system.rotation_axis.direction = tilted_rotation_axis

    A = ProjectionOperator(ig, ag)
    proj = A.direct(grid_tilt)

    recon_tilt = FBP(image_geometry=None, acquisition_geometry=ag)(proj)
    plt.plot(recon_tilt.array[128, :, 128], label=tilt)

plt.xlabel('Horizontal y')
plt.legend()
plt.grid()

Next we try a similar appraoch with a cylinder sample

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from cil.framework import ImageData, ImageGeometry, AcquisitionData, AcquisitionGeometry
from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer
from cil.plugins.astra import ProjectionOperator
from cil.plugins.astra.processors import FBP
from cil.processors import TransmissionAbsorptionConverter, Padder

In [ ]:
import sys, os
from gvxrPython3 import gvxr
from gvxrPython3.JSON2gVXRDataReader import *

In [ ]:
def create_cylinder_with_spheres(simulation_name='cylinder', cylinder_radius = 100, plane = 'xy'):
    
    gvxr.removePolygonMeshesFromSceneGraph()
    sphere_radius =  cylinder_radius/10
    gvxr.makeCylinder(simulation_name, 100, sphere_radius*2, cylinder_radius, "um")

    sphere_spacing = 2 * sphere_radius + (sphere_radius/2)

    n_steps = int((cylinder_radius - sphere_radius) // sphere_spacing)


    i_values = np.arange(-n_steps, n_steps + 1) * sphere_spacing
    j_values = np.arange(-n_steps, n_steps + 1) * sphere_spacing

    
    positions = [
        (i, j)
        for i in i_values
        for j in j_values
        if np.sqrt(i**2 + j**2) <= cylinder_radius - sphere_radius
    ]

    # Precompute translation vectors based on the plane
    if plane == 'xy':
        translations = [(i, j, 0) for i, j in positions]
        gvxr.rotateNode(simulation_name, 90, 1, 0, 0)
        gvxr.applyCurrentLocalTransformation(simulation_name)
    elif plane == 'yz':
        translations = [(0, i, j) for i, j in positions]
        gvxr.rotateNode(simulation_name, 90, 1, 0, 0)
        gvxr.rotateNode(simulation_name, 90, 0, 0, 1)
        gvxr.applyCurrentLocalTransformation(simulation_name)
    elif plane == 'xz':
        translations = [(i, 0, j) for i, j in positions]
    else:
        raise ValueError(f"Unsupported plane: {plane}")

                
    for N, (x, y, z) in enumerate(translations):
        sphere_name = f"sphere_{N}"
        # print(sphere_name)
        gvxr.makeSphere(sphere_name, 50, 50, sphere_radius, "um")
        gvxr.translateNode(sphere_name, x, y, z, "um")
        gvxr.applyCurrentLocalTransformation(sphere_name)
        gvxr.addMesh(simulation_name, sphere_name)

    gvxr.addPolygonMeshAsInnerSurface(simulation_name)
    gvxr.setCompound(simulation_name, "SiO2")
    gvxr.setDensity(simulation_name, 2.2,"g.cm-3")

Now let's try with a more realistic sample. For this example we will use the gVXR (GPU virtual x-ray) package which can be used to create realistic x-ray simulations. We use a gVXR digital twin of the DLS Dual Imaging and Diffraction (DIAD) beamline which can be accessed from https://github.com/TomographicImaging/DIAD2gVXR

In [ ]:
# point to the digital twin code
sys.path.append(os.path.abspath('../DIAD2gVXR/code'))
from DiadModel import DiadModel

# create a digital twin simulation and initialise with some experimental parameters
diad_model = DiadModel()
pixels_x = 500
pixels_y = 500
diad_model.detector_cols = pixels_x
diad_model.detector_rows = pixels_y
diad_model.initSimulationEnegine()
energy_in_keV = 25
exposure_in_sec = 3
diad_model.initExperimentalParameters(1, "m", energy_in_keV, exposure_in_sec)


Create a simulation of a silica cylinder containing spheres, to start we will simulate the cylinder on the yz plane, i.e. orthogonal to the beam along x

In [ ]:
# Create the simulation
simulation_name = "cylinder"
create_cylinder_with_spheres(simulation_name=simulation_name, cylinder_radius=100, plane='yz')
# Compute an X-ray image
gvxr.displayScene()
xray_image = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()
show2D(xray_image)

Simulate a full CT scan for this sample

In [ ]:
# specify number of projections
start = 0
stop = 360
step = 1
angle_set = np.arange(start, stop, step)
xray_image_set = np.zeros((stop, pixels_x, pixels_y), dtype=np.float32)

# specify the rotation axis, around z
rotation_axis = np.array([0, 0, 1])
for N in angle_set:
    # Rotate
    gvxr.rotateNode(simulation_name, N, *rotation_axis)
    # Compute xray image
    xray_image = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()
    xray_image_set[N] = xray_image
    # Rotate back to origin
    gvxr.rotateNode(simulation_name, -N, *rotation_axis)

# use the islicer tool to scroll through the projections
islicer(xray_image_set)

Define the geometry for the reconstruction. 

In CIL the default beam direction is along the y axis and the detector is along x z

In [ ]:
beam_direction = np.array([0, 1, 0])
detector_direction_x = np.array([1, 0, 0])
detector_direction_y = np.array([0, 0, -1])
rotation_axis = np.array([0, 0, 1])

ag = AcquisitionGeometry.create_Parallel3D(ray_direction = beam_direction,
                                      detector_direction_x = detector_direction_x,
                                      detector_direction_y = detector_direction_y,
                                      rotation_axis_direction = rotation_axis)              
ag.set_angles(angle_set)
ag.set_panel((pixels_x, pixels_y))

show_geometry(ag)

data = AcquisitionData(xray_image_set, geometry=ag)
data.reorder('astra')

Apply the Beer-Lambert law to view the absorption

In [ ]:
data = TransmissionAbsorptionConverter(white_level=1.0)(data)
show2D(data, slice_list=[('angle', 0), ('angle', 45), ('angle',90), ('angle', 135), ('angle',180)], num_cols=5)

The attenuation varies a lot in each projection, we can try reconstructing the data.

In [ ]:
# Reconstruct using FBP
recon = FBP(image_geometry=None, acquisition_geometry=ag)(data)

# Plot the results
slice_list = [('vertical','centre'), ('horizontal_y',int(recon.shape[2]/2)), ('horizontal_x',int(recon.shape[1]/2))]
show2D(recon,
       slice_list=slice_list,
       num_cols=3)

The artefacts are much more noticeable here so we try a laminography setup. We choose a tilt angle as small as possible to equalise the attenuation per projection

In [ ]:
# Create the simulation
simulation_name = "cylinder"
create_cylinder_with_spheres(simulation_name=simulation_name, cylinder_radius=100, plane='xy')
# Compute an X-ray image
gvxr.displayScene()
xray_image1 = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()

# tilt the sample
tilt = 15 # degrees
tilt_axis = np.array([0, 1, 0]) # around the detector x direction

gvxr.rotateNode(simulation_name, tilt, *tilt_axis)
xray_image2 = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()
show2D([xray_image1, xray_image2], ['Flat sample', 'Tilted sample'])

Simulate the full CT scan

In [ ]:
# specify number of projections
start = 0
stop = 360
step = 1
angle_set = np.arange(start, stop, step)
xray_image_set = np.zeros((stop, pixels_x, pixels_y), dtype=np.float32)

# specify the rotation axis, around z
rotation_axis = np.array([0, 0, 1])
for N in angle_set:
    # Rotate
    gvxr.rotateNode(simulation_name, N, *rotation_axis)
    # Compute xray image
    xray_image = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()
    xray_image_set[N] = xray_image
    # Rotate back to origin
    gvxr.rotateNode(simulation_name, -N, *rotation_axis)

# use the islicer tool to scroll through the projections
islicer(xray_image_set)

In [ ]:
beam_direction = np.array([0, 1, 0])
detector_x_direction = np.array([1, 0, 0])
detector_y_direction = np.array([0, 0, -1])
rotation_axis = np.array([0, 0, 1]) # the untilted rotation axis

tilt_rad = np.deg2rad(tilt)
rotation_matrix = Rotation.from_rotvec(tilt_rad * detector_x_direction)
tilted_rotation_axis = rotation_matrix.apply(rotation_axis)

ag = AcquisitionGeometry.create_Parallel3D(ray_direction = beam_direction,
                                      detector_direction_x = np.array([1, 0, 0]),
                                      detector_direction_y = np.array([0, 0, -1]),
                                      rotation_axis_direction = list(tilted_rotation_axis))                   
ag.set_angles(angle_set)
ag.set_panel((pixels_x, pixels_y),
             list([diad_model.effective_pixel_spacing_in_um[0]/1000, diad_model.effective_pixel_spacing_in_um[0]/1000]))

show_geometry(ag)

data = AcquisitionData(xray_image_set, geometry=ag)
data.reorder('astra')

In [ ]:
# apply Beer-Lambert law
data = TransmissionAbsorptionConverter(white_level=1.0)(data)
show2D(data, slice_list=[('angle', 0), ('angle', 45), ('angle',90), ('angle', 135), ('angle',180)], num_cols=5)

In [ ]:
# Reconstruct using FBP
recon = FBP(image_geometry=None, acquisition_geometry=ag)(data)

# Plot the results
slice_list = [('vertical','centre'), ('horizontal_y',int(recon.shape[2]/2)), ('horizontal_x',int(recon.shape[1]/2))]
show2D(recon,
       slice_list=slice_list,
       num_cols=3)

Another common scenario with laminography samples, is that only a small region of interest (ROI) within the sample is fully scanned and parts of the sample outside the ROI will come in and out of view depending on the projection. We can simulate this scenario by generating a larger cylinder

In [ ]:
# Create the simulation
simulation_name = "cylinder"
create_cylinder_with_spheres(simulation_name=simulation_name, cylinder_radius=200, plane='xy')
# Compute an X-ray image
gvxr.displayScene()
xray_image1 = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()

# tilt the sample
tilt = 15 # degrees
tilt_axis = np.array([0, 1, 0]) # around the detector x direction

gvxr.rotateNode(simulation_name, tilt, *tilt_axis)
xray_image2 = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()
show2D([xray_image1, xray_image2], ['Flat sample', 'Tilted sample'])

Simulate the CT scan, convert to absorption and create the geometry

In [ ]:
# specify number of projections
start = 0
stop = 360
step = 1
angle_set = np.arange(start, stop, step)
xray_image_set = np.zeros((stop, pixels_x, pixels_y), dtype=np.float32)

# specify the rotation axis, around z
rotation_axis = np.array([0, 0, 1])
for N in angle_set:
    # Rotate
    gvxr.rotateNode(simulation_name, N, *rotation_axis)
    # Compute xray image
    xray_image = np.array(gvxr.computeXRayImage(), dtype=np.single)/ gvxr.getTotalEnergyWithDetectorResponse()
    xray_image_set[N] = xray_image
    # Rotate back to origin
    gvxr.rotateNode(simulation_name, -N, *rotation_axis)

# create the geometry
beam_direction = np.array([0, 1, 0])
detector_x_direction = np.array([1, 0, 0])
detector_y_direction = np.array([0, 0, -1])
rotation_axis = np.array([0, 0, 1]) # the untilted rotation axis

tilt_rad = np.deg2rad(tilt)
rotation_matrix = Rotation.from_rotvec(tilt_rad * detector_x_direction)
tilted_rotation_axis = rotation_matrix.apply(rotation_axis)

ag = AcquisitionGeometry.create_Parallel3D(ray_direction = beam_direction,
                                      detector_direction_x = np.array([1, 0, 0]),
                                      detector_direction_y = np.array([0, 0, -1]),
                                      rotation_axis_direction = list(tilted_rotation_axis))                   
ag.set_angles(angle_set)
ag.set_panel((pixels_x, pixels_y),
             list([diad_model.effective_pixel_spacing_in_um[0]/1000, diad_model.effective_pixel_spacing_in_um[0]/1000]))

data = AcquisitionData(xray_image_set, geometry=ag,)
data.reorder('astra')
data = TransmissionAbsorptionConverter(white_level=1.0)(data)
show2D(data, slice_list=[('angle', 0), ('angle', 45), ('angle',90), ('angle', 135), ('angle',180)], num_cols=5)

In [ ]:
# Reconstruct using FBP
recon = FBP(image_geometry=None, acquisition_geometry=ag)(data)

# Plot the results
slice_list = [('vertical','centre'), ('horizontal_y',int(recon.shape[2]/2)), ('horizontal_x',int(recon.shape[1]/2))]
show2D(recon,
       slice_list=slice_list,
       num_cols=3)

The reconstruction contains a bright ring, this is common with ROI data because the projections contain information from outside the ROI. To solve this we need to define an acquisition geometry which is larger than the detector to account for the information outside the ROI.

In [ ]:
# Reconstruct using FBP
ig = ag.get_ImageGeometry()
padsize = 100 # approximately the size of the object outside the detector (in pixels)
data = Padder.edge(pad_width={'horizontal': padsize})(data)

recon = FBP(image_geometry=ig, acquisition_geometry=data.geometry)(data)

# Plot the results
slice_list = [('vertical','centre'), ('horizontal_y',int(recon.shape[2]/2)), ('horizontal_x',int(recon.shape[1]/2))]
show2D(recon,
       slice_list=slice_list,
       num_cols=3)

In [ ]:
gvxr.destroy()